In [ ]:
!pip install boto3

In [2]:
import boto3
import time
from google.colab import userdata

In [3]:
# Crie uma sessao usando as credenciais do usuario IAM
session = boto3.Session(
    aws_access_key_id = userdata.get('aws_access_key_id'),
    aws_secret_access_key = userdata.get('aws_secret_access_key'),
    region_name = 'us-east-2'
)

In [4]:
# Crie um cliente SQS
service_sqs = session.client('sqs')
sqs_queue_name = 'eda-sqs-msg.fifo'

In [ ]:
# Crie uma fila
response = service_sqs.create_queue(
    QueueName = sqs_queue_name,
    Attributes={
        'FifoQueue': 'true',
        'ContentBasedDeduplication': 'true',
        'DelaySeconds': '5',  # Atraso de 5 segundos para as mensagens (opcional)
        'MessageRetentionPeriod': '86400'  # Retencao de mensagens por 1 dia (opcional)
    }
)

# Imprima a URL da fila criada
print(response['QueueUrl'])

In [6]:
# Obtenha a URL da fila
response = service_sqs.get_queue_url(QueueName = sqs_queue_name)
queue_url = response['QueueUrl']  # URL da fila criada

In [ ]:
queue_sequence = 1

while True:
    message_body = "Mensagem de teste numero " + str(queue_sequence) + " em tempo real"  # Conteudo da mensagem
    message_group_id = 'sql-google-colab'  # ID do grupo de mensagens (opcional)
    message_deduplication_id = str(time.time())  # ID de deduplicacao (opcional)
    message_attributes = {
        'Projeto': {
            'DataType': 'String',
            'StringValue': 'EDA'
        },
        'DataVersao': {
            'DataType': 'Number',
            'StringValue': '20240731'
        }
    }

    response = service_sqs.send_message(
        QueueUrl=queue_url,
        MessageBody=message_body,
        MessageAttributes=message_attributes,
        MessageGroupId=message_group_id,  # ID do grupo de mensagens
        MessageDeduplicationId=message_deduplication_id  # ID de deduplicacao
    )

    print("Mensagem ID:", response['MessageId'], "| Texto Mensagem:", message_body)

    queue_sequence += 1

    time.sleep(5)  # Aguarda 5 segundos antes de enviar a proxima mensagem

In [ ]:
# Exclua a fila
service_sqs.delete_queue(QueueUrl=queue_url)

print('Fila excluida com sucesso.')